In [1]:
import os
import sys
from langchain_llms import SambaStudio, Sambaverse, SambaNovaFastAPI
from langchain_embeddings import SambaStudioEmbeddings
from dotenv import load_dotenv
import json

current_dir = os.getcwd()
utils_dir = os.path.abspath(os.path.join(current_dir, ".."))
repo_dir = os.path.abspath(os.path.join(utils_dir, ".."))

load_dotenv(os.path.join(repo_dir,".env"))


True

# SambaStudio endpoint

## Non streaming

In [2]:

llm = SambaStudio(
        streaming=False,
        #base_uri="api/predict/generic",
        model_kwargs={
            "do_sample": True, 
            "temperature": 0.01,
            "max_tokens_to_generate": 256,
            "process_prompt": False,
            "select_expert": "llama-2-7b-chat-hf",
            # "repetition_penalty": 1.0,
            # "top_k": 50",
            # "top_logprobs": 0,
            # "top_p": 1.0
        }
    ) 


In [3]:
llm.invoke("tell me a 50 word tale")

'of a character who is a master of disguise\n\nSure! Here is a 50-word tale of a character who is a master of disguise:\n\n"Araxys, the skilled disguise artist, transformed into a stunning mermaid to infiltrate a pirate\'s lair, then morphed into a loyal servant to gain the trust of the pirate\'s crew."'

## Streaming

In [4]:
llm=SambaStudio(
        streaming=True,
        model_kwargs={
            "do_sample": True,
            "max_tokens_to_generate": 256,
            "temperature": 0.01,
            "process_prompt": False,
            "select_expert": "llama-2-7b-chat-hf",
            # "repetition_penalty": 1.0,
            # "top_k": 50",
            # "top_logprobs": 0,
            # "top_p": 1.0
            }
        )

In [5]:
for chunk in llm.stream("tell me a 50 word tale"):
    print(chunk, end="", flush=True)

 of a character who is a master of disguise

Sure! Here is a 50-word tale of a character who is a master of disguise:

"Araxys, the skilled disguise artist, transformed into a stunning mermaid to infiltrate a pirate's lair. With a flick of her tail, she charmed the pirates and stole their treasure."

# Sambaverse


## Non Streaming

In [6]:
llm = Sambaverse(
    sambaverse_model_name = "Meta/llama-2-7b-chat-hf",
    model_kwargs={
        "do_sample": True, 
        "max_tokens_to_generate": 256,
        "temperature": 0.01,
        "process_prompt": False,
        "select_expert": "llama-2-7b-chat-hf",
        #"stop_sequences": '\"sequence1\",\"sequence2\"',
        #"repetition_penalty":  1.0,
        #"top_k": 50,
        #"top_p": 1.0
        }
    ) 

In [7]:
llm.invoke("tell me a 50 word tale")

'of a character who is a master of disguise\n\nSure! Here is a 50-word tale of a character who is a master of disguise:\n\n"Araxys, the skilled disguise artist, transformed into a stunning mermaid to infiltrate a pirate\'s lair, then morphed into a loyal servant to gain the trust of the pirate\'s crew."'

## Streaming

In [8]:
llm = Sambaverse(
    streaming=True,
    sambaverse_model_name="Meta/llama-2-7b-chat-hf",
    model_kwargs={
        "do_sample": True, 
        "max_tokens_to_generate": 256,
        "temperature": 0.01,
        "process_prompt": False,
        "select_expert": "llama-2-7b-chat-hf",
        #"stop_sequences": '\"sequence1\",\"sequence2\"',
        #"repetition_penalty":  1.0,
        #"top_k": 50,
        #"top_p": 1.0
        }
    ) 

In [9]:

for chunk in llm.stream("tell me a 50 word tale"):
    print(chunk, end="", flush=True)

 of a character who is a master of disguise

Sure! Here is a 50-word tale of a character who is a master of disguise:

"Araxys, the skilled disguise artist, transformed into a stunning mermaid to infiltrate a pirate's lair. With a flick of her tail, she charmed the pirates and stole their treasure."

# FastCoe endpoint

In [2]:
llm = SambaNovaFastAPI(model='llama3-8b')


In [3]:
import json
llm.invoke(json.dumps([{'role': 'user', 'content': 'hello'}]))

0


ValueError: Error raised by the inference endpoint: Error raised by the inference endpoint: Sambanova /complete call failed with status code 503.<html>
<head><title>503 Service Temporarily Unavailable</title></head>
<body>
<center><h1>503 Service Temporarily Unavailable</h1></center>
<hr><center>nginx</center>
</body>
</html>
.

In [4]:
llm.invoke('hello')

"Hello! It's nice to meet you. Is there something I can help you with, or would you like to chat?"

In [5]:
for i in llm.stream('hello tell me a long story'):
    print(i)


I'd be delighted to spin you a yarn! Here's a long story for you:

Once upon 
a time, in a small village nestled in the rolling hills of Provence, France, there lived 
a young girl named Sophie. Sophie was a curious and adventurous soul, with a mop of curly brown 
hair and a smile that could light up the darkest of rooms. She lived with her parents, Pierre 
and Colette, in a charming stone cottage on the outskirts of the village, surrounded by fields of 
lavender and sunflowers.

Sophie's parents were kind and loving, but they were also very 
traditional. They ran a small bakery in the village, where they made the most delicious bread and pastries 
in all the land. Sophie loved helping out in the bakery, learning the secrets of the trade from 
her parents and experimenting with new recipes of her own.

One day, when Sophie was 12 years 
old, a strange and mysterious stranger arrived in the village. He was tall and gaunt, with 
piercing green eyes and a wild shock of white hair. He wor

# Embeddings

In [ ]:
embedding = SambaStudioEmbeddings(
    batch_size=1,
    model_kwargs = {
    "select_expert":"e5-mistral-7b-instruct"
    })
embedding.embed_documents(["tell me a 50 word tale","tell me a joke"])
embedding.embed_query("tell me a 50 word tale")

In [13]:
from langchain.vectorstores import Chroma
from langchain.schema import Document

docs=["tell me a 50 word tale","tell me a joke","when was America discoverd?", "how to build an engine?", "give me 3 party activities", "give me three healty dishes"]
docs=[Document(doc) for doc in docs]

query = "prompt for generating something fun"

vectordb = Chroma.from_documents(docs, embedding)
retriever = vectordb.as_retriever()

retriever.get_relevant_documents(query)


/Users/jorgep/Documents/ask_public_own/finetuning_env/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


[Document(page_content='tell me a 50 word tale'),
 Document(page_content='tell me a joke'),
 Document(page_content='give me 3 party activities'),
 Document(page_content='give me three healty dishes')]